In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
np.set_printoptions(4, suppress=True)
torch.set_printoptions(4, sci_mode=False)
import sys
sys.path.append('../')
from ttgo import TTGO
from utils import test_ttgo
from ur10_kinematics import Ur10Kinematics
from manipulator_utils import dist_orientation_fixed
device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cpu


In [2]:
trained_model = torch.load('ur10_ik.pickle', map_location=torch.device('cpu'))
b_goal,b_orient = trained_model['b']
d0_theta = trained_model['d0_theta']
dh_x = trained_model['dh_x']
domain = trained_model['domain']
Rd_0 = trained_model['Rd_0']
test_task = trained_model['test_task']

In [3]:
# Setup the robot and the environment

ur10 = Ur10Kinematics(device=device)
n_joints= ur10.n_joints
# Desired orientation (fixed orientation)

def cost_all(x): # For inverse kinematics
    x = x.to(device)
    batch_size = x.shape[0]
    goal_loc = x[:,:3]
    q = x[:,3:]  # batch x joint angles
    _, end_loc, end_R = ur10.forward_kin(q) # batch x joint x keys x positions
    # cost on error in end-effector position
    d_goal = torch.linalg.norm(end_loc-goal_loc, dim=1)
    # cost on error in end-effector orientation
    d_orient = dist_orientation_fixed(Rd_0,end_R,device=device)
    c_total = 0.5*(d_goal/b_goal + d_orient/b_orient)
    c_return = torch.cat((c_total.view(-1,1), d_goal.view(-1,1), d_orient.view(-1,1)),dim=-1)
    return c_return

def cost(x):
    return cost_all(x)[:,0]


def pdf(x):
    x = x.to(device)
    pdf_ = torch.exp(-cost(x)**2) 
    return pdf_

#####################################################################


In [4]:

ttgo = TTGO(domain=domain,pdf=pdf,cost=cost)
ttgo.tt_model = trained_model['tt_model']
ttgo.to('cpu')

# Prepare for the task
sites_task = list(range(3))
ttgo.set_sites(sites_task)

In [5]:
Rd_0

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

## Visualization

In [6]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial
# import the environment (SDF and for graphics visualization in pybullet)
import sys
sys.path.append('../../lib')

import sys
DATA_PATH = './data'
robot_urdf = DATA_PATH + '/urdf/ur10/ur10.urdf'


physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

p.resetSimulation()

# for i in range(8):
#     print(i, p.getJointInfo(robot_id, i)[1])
    
robot_id = p.loadURDF(robot_urdf)

dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(1,7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
set_q_std = partial(set_q,robot_id, pb_joint_indices)

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,0.), (0,0,0,1))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

ee_pb_id = 7


In [7]:
# # Prepare for the task
# sites_task = list(range(3))
# ttgo.set_sites(sites_task)

In [8]:
s = np.random.randint(0,test_task.shape[0]-1)
sample_xe = torch.tensor([-0.0, 0.3, 0.7]) #
print(sample_xe)


n_solutions=50
n_samples_tt = 200 #50*n_solutions
n_samples_rand= 1*n_samples_tt

alpha=0.75; norm=1 ; sample_replace = True; 

t1 = time.time()
samples_tt, samples_idx = ttgo.sample(n_samples=n_samples_tt, x_task=sample_xe.reshape(1,-1),alpha=alpha, norm=norm, sample_replace=sample_replace)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],results= ttgo.optimize(state,bound=True)
t2 = time.time()

# samples_rand, _ = ttgo.sample_random(n_samples=n_samples_rand,  x_task=sample_xe.reshape(1,-1))
# state_k_rand = ttgo.choose_top_k_sample(samples_rand,n_solutions)

# #Optimize
# state_k_rand_opt = state_k_rand*1
# for i, state in enumerate(state_k_rand):
#     state_k_rand_opt[i,:],results= ttgo.optimize(state,bound=True)
t3=time.time()

print("Time taken:", (t2-t1), t3-t2)
             
c_tt =  cost_all(state_k_tt_opt)
# c_rand =   cost_all(state_k_rand_opt)

print("Cost-mean-tt:",torch.mean(c_tt,dim=0))
# print("Cost-mean-rand:",torch.mean(c_rand,dim=0))



tensor([-0.00, 0.30, 0.70])
Time taken: 7.0895466804504395 0.00010991096496582031
Cost-mean-tt: tensor([    0.00,     0.00,     0.00])


In [9]:
x_target = sample_xe[:3].numpy()
joint_angles_k = state_k_tt[:,3:].numpy() 
joint_angles_k_opt = state_k_tt_opt[:,3:].numpy() 

In [ ]:
# _,_,test_sphere = create_primitives(p.GEOM_SPHERE, radius = 0.02)
# p.resetBasePositionAndOrientation(test_sphere, (0,0,1.), (0,0,0,1))
_ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
pos = x_target[:]

p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


k = joint_angles_k.shape[0]
dt = 0.5
dT = 2
for i in range(2*k):
    set_q_std(joint_angles_k[i%k])
    time.sleep(dt)
    set_q_std(joint_angles_k_opt[i%k])
    time.sleep(2*dt)
    